In [2]:
# basic data analytics
import pandas as pd
import numpy as np
import sklearn
import pickle

# nlp modules
import nltk
import spacy
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

import multiprocessing
import string

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfTransformer, TfidfVectorizer


import gensim
from gensim import corpora, models
from gensim.models.ldamulticore import LdaMulticore
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()
import warnings

# warnings.simplefilter('once')
warnings.simplefilter('ignore')

num_processors = multiprocessing.cpu_count()
num_processors

workers = num_processors-1

print(f'Using {workers} workers')

Using 7 workers


In [3]:
df = pd.read_csv('/home/jupyter/df_news.csv', lineterminator='\n')

In [4]:
df = df.drop('Unnamed: 0', axis = 1)

In [5]:
df = df.drop(['text_notitle', 'split', 'split_len'], axis=1)

In [6]:
df.head(3)

,url,date,language,title,text,text_clean
0,http://en.people.cn/n3/2021/0318/c90000-983012...,2021-03-18,en,Artificial intelligence improves parking effic...,\n\nArtificial intelligence improves parking e...,thanks to the application of an artificial int...
1,http://newsparliament.com/2020/02/27/children-...,2020-02-27,en,Children With Autism Saw Their Learning and So...,\nChildren With Autism Saw Their Learning and ...,scientists who designed an artificially clever...
2,http://www.dataweek.co.za/12835r,2021-03-26,en,"Forget ML, AI and Industry 4.0 – obsolescence ...","\n\nForget ML, AI and Industry 4.0 – obsolesce...",the world entered a new era of accelerated tra...


In [19]:
doc_complete = df.head(200)['text_clean'].values.tolist()

# def clean(doc):
#     all_str = [str(i) for i in doc_complete]
#     length_long =  [i for i in doc_complete if len(i) > 100]
#     stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
#     punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
#     normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
#     return normalized

# documents = [clean(doc).split() for doc in doc_complete]  

# [type(doc) for doc in doc_complete]

doc_complete

['in this report , the artificial intelligence software market is valued at usd xx million in 2017 and is expected to reach usd xx million by the end of 2025 , growing at a cagr of xx % between 20 . artificial intelligence software market has been broken down by major regions , with complete market estimates on the basis of products/applications on a regional basis . browse full research report at https : //www.crystalmarketreport.com/2018-2025-artificial-intelligence-software-report-on-global-and-united-states-market-status-and-forecast-by-players-types-and-applications this report studies the global artificial intelligence software market , analyzes and researches the artificial intelligence software development status and forecast in united states , eu , japan , china , india and southeast asia . this report focuses on the top players in global market , like cortana ( microsoft ) google assistant siri braina h2o alphago opennn a.l.i.c.e ( artificial linguistic internet computer enti

In [23]:
%%time
# Define the number of topics
num_topics = 10

# Preprocess your news articles and create a list of documents
# Each document should be a list of tokens (words) representing an article

stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()

doc_complete = df.head(200)['text_clean'].values.tolist()

def clean(doc):
    all_str = [i for i in doc if type(i) == 'str']
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    return normalized

documents = [clean(doc).split() for doc in doc_complete]     



# Create a dictionary from the preprocessed documents
dictionary = corpora.Dictionary(documents)

# Convert the dictionary into a bag-of-words representation
corpus = [dictionary.doc2bow(doc) for doc in documents]

# Perform LDA modeling using multicore implementation
lda_model = LdaMulticore(corpus=corpus, num_topics=num_topics, id2word=dictionary, workers=workers)

# Print the most contributing words for each topic
for topic in lda_model.print_topics(num_topics=num_topics, num_words=10):
    print(topic)

TypeError: 'float' object is not iterable

In [38]:
len(documents)

100

In [32]:
from gensim.models import CoherenceModel

# Calculate coherence score
coherence_model = CoherenceModel(model=lda_model, texts=documents, dictionary=dictionary, coherence='c_v')
coherence_score = coherence_model.get_coherence()

# Print the coherence score
print("Coherence Score:", coherence_score)


Coherence Score: 0.3524630835654374
